# دانلود فایل تلگرام و ذخیره مستقیم در Google Drive

این نوت‌بوک برای کاربران فارسی‌زبان (بدون نیاز به برنامه‌نویسی) آماده شده است.

## 1) نصب کتابخانه‌های لازم
این سلول را اجرا کنید.

In [ ]:
!pip -q install telethon nest_asyncio

## 2) اتصال Google Drive
بعد از اجرا، روی لینک کلیک کنید و مجوز دسترسی بدهید.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 3) تنظیم اطلاعات تلگرام
- API ID و API HASH را از https://core.telegram.org/api/obtaining_api_id بگیرید.
- channel می‌تواند آیدی کانال (مثل `@example_channel`) باشد.
- message_id همان عدد انتهای لینک پیام است.

In [ ]:
api_id = int(input('API ID: '))
api_hash = input('API HASH: ').strip()
phone = input('Phone (example: +98912...): ').strip()
channel = input('Channel username/id (example: @mychannel): ').strip()
message_id = int(input('Message ID: '))
download_dir = '/content/drive/MyDrive/TelegramDownloads'

## 4) ورود به تلگرام و دانلود فایل
اگر تایید دومرحله‌ای فعال باشد، پسورد از شما پرسیده می‌شود.

In [ ]:
import os
import asyncio
import nest_asyncio
from getpass import getpass
from telethon import TelegramClient
from telethon.errors import SessionPasswordNeededError

nest_asyncio.apply()
os.makedirs(download_dir, exist_ok=True)

async def main():
    client = TelegramClient('colab_session', api_id, api_hash)
    await client.connect()

    if not await client.is_user_authorized():
        await client.send_code_request(phone)
        code = input('کد ارسال‌شده توسط تلگرام را وارد کنید: ').strip()
        try:
            await client.sign_in(phone=phone, code=code)
        except SessionPasswordNeededError:
            password = getpass('پسورد تایید دومرحله‌ای: ')
            await client.sign_in(password=password)

    message = await client.get_messages(entity=channel, ids=message_id)
    if not message:
        raise ValueError('پیام پیدا نشد. channel و message_id را بررسی کنید.')

    if not message.media:
        raise ValueError('این پیام فایل/مدیا ندارد.')

    print('شروع دانلود...')
    saved_path = await client.download_media(message, file=download_dir)
    print(f'✅ دانلود کامل شد. مسیر فایل: {saved_path}')

    await client.disconnect()

await main()

## 5) خروجی
فایل دانلودی را داخل مسیر زیر در Google Drive پیدا می‌کنید:
`MyDrive/TelegramDownloads`